In [35]:
import logging

import tensorflow as tf
import os
import json
import sys
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

from eoflow.models.segmentation_unets import ResUnetA

logging.getLogger().setLevel(logging.ERROR)

In [10]:
def visualization_parcels(data, label, num_plt=5, beta=0.3, alpha=0.3, label_="GT", ind_s=0, num_h=1):
    #fig = plt.figure(figsize=(15, 12))
    plt.subplot(num_h, num_plt, ind_s+1)
    bgr = data['features'][0, :, :, :3].numpy() # with normalization
    rgb = bgr[..., [2, 1, 0]]
    plt.imshow(rgb)
    plt.title('RGB')
    plt.subplot(num_h, num_plt, ind_s+2)
    extent = np.argmax(label['extent'], axis=-1)[0]
    plt.imshow(extent, cmap='gray')
    plt.title(f'{label_} extent')
    plt.subplot(num_h, num_plt, ind_s+3)
    boundary = np.argmax(label['boundary'], axis=-1)[0]
    plt.imshow(boundary, cmap='gray')
    plt.title(f'{label_} boundary')
    blended_image = (1 - alpha) * rgb + alpha * extent[:, :, np.newaxis]
    plt.subplot(num_h, num_plt, ind_s+4)
    plt.imshow(blended_image)
    plt.title(f'{label_} blend RGB & extent')
    blended_image = (1 - beta) * rgb + beta * boundary[:, :, np.newaxis]
    plt.subplot(num_h, num_plt, ind_s+5)
    plt.imshow(blended_image)
    plt.title(f'{label_} blend RGB & boundary')

In [30]:
chkpt_folder = "checkpoints" # change to your own
model_cfg_path = "model_cfg.json" # change to your own

In [ ]:
input_shape = (256, 256, 4)
input_shape = dict(features=[None, *input_shape])
with open(model_cfg_path, 'r') as jfile:
    model_cfg = json.load(jfile)
# initialise model from config, build, compile and load trained weights
model = ResUnetA(model_cfg)
model.build(input_shape)
model.net.compile()
model.net.load_weights(f'{chkpt_folder}/model.ckpt')

In [13]:
model.net.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 features (InputLayer)          [(None, 256, 256, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  256         ['features[0][0]']               
                                )                                                                 
                                                                                                  
 res_conv2d (ResConv2D)         (None, 256, 256, 32  73728       ['conv2d[0][0]']                 
                                )                                                             

In [32]:
dataset_path = "training_data/datasets/test" # change to your own
batch_size = 1
dataset = tf.data.Dataset.load(dataset_path)
print(dataset)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

<_LoadDataset element_spec=({'features': TensorSpec(shape=(256, 256, 4), dtype=tf.float32, name=None)}, {'distance': TensorSpec(shape=(256, 256, 2), dtype=tf.float32, name=None), 'boundary': TensorSpec(shape=(256, 256, 2), dtype=tf.float32, name=None), 'extent': TensorSpec(shape=(256, 256, 2), dtype=tf.float32, name=None)})>


In [33]:
path_to_pdf = 'some.pdf' # change to your own

In [ ]:
# visualize predictions
with PdfPages(path_to_pdf) as pdf: # diff normalization params 
    for data, label in dataset: # as_numpy_iterator
        pred = model.net.predict(data)
        pred_l = {"distance": pred[0], "boundary": pred[1], "extent": pred[2]}
        fig = plt.figure(figsize=(15, 7))
        visualization_parcels(data, label, label_='GT', num_h=2, ind_s=0)
        visualization_parcels(data, pred_l, label_='pred', num_h=2, ind_s=5)
        plt.show()
        pdf.savefig(fig) 
        plt.close()